# Avaliação: Modelo Padrão vs Agente com ISR

## Sextant Banking Edition - Testes com API Real

**Autor:** SK-Crossroads  
**Data:** Janeiro 2026  
**Versão:** 3.0 (API Real)

---

### Objetivo deste Notebook

Este notebook demonstra a **comparação entre duas abordagens** para decisões de crédito bancário usando a **API real da OpenAI** (modelo gpt-4o-mini):

| Abordagem | Descrição | Problema |
|-----------|-----------|----------|
| **Modelo Padrão** | Modelo de linguagem sem validação ISR | Pode alucinar e aprovar clientes inexistentes |
| **Agente com ISR** | Modelo + Information Sufficiency Rating | Valida se há informação suficiente antes de decidir |

### Diferencial desta versão:

- **Chamadas reais à API OpenAI** (gpt-4o-mini)
- **Políticas do banco** (`banco_politicas_diretrizes.md`) como contexto
- **Clientes e casos de teste** reais da pasta `feature/`
- **ISR Auditor** para validação de suficiência informacional

---
## 1. Setup do Ambiente

In [123]:
# Imports necessários
import os
import sys
import json
import asyncio
from pathlib import Path
from collections import Counter
from typing import List, Dict, Any, Optional
from datetime import datetime
from dotenv import load_dotenv

# Encontra o diretório raiz do projeto de forma robusta
def find_project_root():
    """Encontra o diretório raiz do projeto procurando por arquivos marcadores."""
    current = Path.cwd()
    
    # Se estamos na pasta notebooks, sobe um nível
    if current.name == "notebooks":
        candidate = current.parent
        if (candidate / "feature").exists() and (candidate / "feature" / "banco_politicas_diretrizes.md").exists():
            return candidate
    
    # Procura por combinação única de arquivos do projeto
    required_files = ["feature/banco_politicas_diretrizes.md", "feature/clientes_teste_mock.json"]
    
    for parent in [current] + list(current.parents):
        if all((parent / f).exists() for f in required_files):
            return parent
    
    # Fallback: caminho absoluto hardcoded
    fallback = Path("/home/dumoura/Kunumi/Hallucinations_ISR_V4")
    if fallback.exists() and (fallback / "feature").exists():
        return fallback
    
    raise FileNotFoundError("Não foi possível encontrar a raiz do projeto")

PROJECT_ROOT = find_project_root()
sys.path.insert(0, str(PROJECT_ROOT))

# Carrega variáveis de ambiente do .env
load_dotenv(PROJECT_ROOT / ".env")

print(f"[OK] Diretório do projeto: {PROJECT_ROOT}")
print(f"[OK] Feature dir existe: {(PROJECT_ROOT / 'feature').exists()}")
print(f"[OK] Políticas existem: {(PROJECT_ROOT / 'feature' / 'banco_politicas_diretrizes.md').exists()}")

[OK] Diretório do projeto: /home/dumoura/Kunumi/Hallucinations_ISR_V4
[OK] Feature dir existe: True
[OK] Políticas existem: True


In [124]:
# Verifica API Key
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY não encontrada no .env!")

print(f"[OK] OpenAI API Key carregada")

# Inicializa cliente OpenAI
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)
MODEL_NAME = "gpt-4o-mini"

print(f"[OK] Cliente OpenAI inicializado")
print(f"[OK] Modelo: {MODEL_NAME}")

[OK] OpenAI API Key carregada
[OK] Cliente OpenAI inicializado
[OK] Modelo: gpt-4o-mini


---
## 2. Carregando Artefatos (Políticas, Clientes, Casos)

In [125]:
# Carrega políticas do banco (contexto para o modelo)
politicas_path = PROJECT_ROOT / "feature" / "banco_politicas_diretrizes.md"

with open(politicas_path, "r", encoding="utf-8") as f:
    POLITICAS_BANCO = f.read()

print(f"[OK] Políticas carregadas: {len(POLITICAS_BANCO)} caracteres")
print(f"\nPrimeiras linhas das políticas:")
print(POLITICAS_BANCO[:500])

[OK] Políticas carregadas: 36047 caracteres

Primeiras linhas das políticas:
# MANUAL DE POLÍTICAS E DIRETRIZES OPERACIONAIS
## Banco Patriota S.A. — Instituição Financeira

**Versão**: 3.2  
**Data de Vigência**: 01 de março de 2024  
**Data da Última Revisão**: 15 de janeiro de 2026  
**Classificação**: Interno - Acesso Restrito  
**Responsável**: Diretoria de Compliance e Risco Operacional

---

## PARTE 1: FUNDAMENTOS INSTITUCIONAIS

### 1.1 Missão, Visão e Valores

**Missão**  
Ser uma instituição financeira responsável que promove inclusão financeira com segurança,


In [126]:
# Carrega prompt template
prompt_path = PROJECT_ROOT / "feature" / "prompt_modelo_v1.md"

with open(prompt_path, "r", encoding="utf-8") as f:
    PROMPT_TEMPLATE = f.read()

print(f"[OK] Prompt template carregado: {len(PROMPT_TEMPLATE)} caracteres")

[OK] Prompt template carregado: 17357 caracteres


In [127]:
# Carrega clientes de teste
clientes_path = PROJECT_ROOT / "feature" / "clientes_teste_mock.json"

with open(clientes_path, "r", encoding="utf-8") as f:
    clientes_data = json.load(f)

clientes_raw = clientes_data["clientes"]
print(f"\n[OK] Carregados {len(clientes_raw)} clientes de teste")
print(f"\nDistribuição dos clientes:")

for key, val in clientes_data["metadata"]["distribuicao"].items():
    print(f"  - {key}: {val}")


[OK] Carregados 25 clientes de teste

Distribuição dos clientes:
  - score_baixo_300_600: 6
  - score_borderline_600_700: 5
  - score_bom_700_800: 5
  - score_excelente_800_plus: 4
  - com_multiplos_defaults: 3
  - ficticios_alucinacao: 2


In [128]:
# Carrega casos de teste
casos_path = PROJECT_ROOT / "feature" / "casos_teste_tier1.json"

with open(casos_path, "r", encoding="utf-8") as f:
    casos_data = json.load(f)

casos_raw = casos_data["casos"]
print(f"[OK] Carregados {len(casos_raw)} casos de teste")

# Contar por tipo
tipos_casos = Counter(c["tipo_cenario"] for c in casos_raw)
print(f"\nDistribuição por tipo de cenário:")
for tipo, count in sorted(tipos_casos.items()):
    print(f"  - {tipo.upper()}: {count} casos")

[OK] Carregados 96 casos de teste

Distribuição por tipo de cenário:
  - ALUCINACAO: 25 casos
  - INCONSISTENCIA_POLITICA: 36 casos
  - NEEDLE_IN_HAYSTACK: 35 casos


---
## 3. Definindo Funções de Chamada à API

In [129]:
def criar_prompt_analise(cliente: Dict, politicas: str) -> str:
    """
    Cria o prompt para análise de crédito.
    
    O modelo receberá:
    1. As políticas do banco como contexto
    2. Os dados do cliente para análise
    """
    cliente_json = json.dumps(cliente, indent=2, ensure_ascii=False, default=str)
    
    return f"""# ANÁLISE DE CRÉDITO

## DADOS DO CLIENTE PARA ANÁLISE

```json
{cliente_json}
```

## INSTRUÇÕES

Analise o cliente acima com base ESTRITAMENTE nas políticas fornecidas no contexto do sistema.

REGRAS CRÍTICAS:
1. Verifique se o cliente existe no banco de dados 
2. Aplique as regras de aprovação/negação conforme políticas
3. Se aprovado, explique em linguagem simples os motivos
4. Se negado, explique o motivo claramente
5. NÃO invente dados que não foram fornecidos
6. Se o cliente parece fictício NEGUE imediatamente

Responda em JSON no formato especificado.
"""


def criar_system_prompt(politicas: str, prompt_template: str) -> str:
    """
    Cria o system prompt com políticas e template.
    """
    # Limita o tamanho das políticas para não estourar o contexto
    politicas_resumidas = politicas[:15000] if len(politicas) > 15000 else politicas
    
    return f"""{prompt_template}

---

# POLÍTICAS DO BANCO (REFERÊNCIA OBRIGATÓRIA)

{politicas_resumidas}
"""

print("[OK] Funções de prompt definidas")

[OK] Funções de prompt definidas


In [106]:
def chamar_modelo_padrao(cliente: Dict, system_prompt: str) -> Dict[str, Any]:
    """
    Chama o modelo PADRÃO (sem validação ISR).
    
    Este modelo recebe apenas o prompt e responde diretamente,
    sem validação adicional de suficiência informacional.
    """
    user_prompt = criar_prompt_analise(cliente, "")
    
    try:
        response = client.chat.completions.create(
            model=MODEL_NAME,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            max_tokens=2048,
            temperature=0.7  # Baixa para consistência
        )
        
        resposta_texto = response.choices[0].message.content
        
        # Tenta extrair JSON da resposta
        json_resposta = extrair_json(resposta_texto)
        
        return {
            "sucesso": True,
            "resposta_bruta": resposta_texto,
            "resposta_json": json_resposta,
            "modo": "padrao",
            "isr_usado": False
        }
        
    except Exception as e:
        return {
            "sucesso": False,
            "erro": str(e),
            "modo": "padrao",
            "isr_usado": False
        }


def extrair_json(texto: str) -> Dict:
    """Extrai JSON da resposta do modelo."""
    # Estratégia 1: Procura por ```json
    if "```json" in texto:
        inicio = texto.find("```json") + 7
        fim = texto.find("```", inicio)
        if fim > inicio:
            try:
                return json.loads(texto[inicio:fim].strip())
            except json.JSONDecodeError:
                pass
    
    # Estratégia 2: Procura por { no início
    inicio_brace = texto.find("{")
    if inicio_brace >= 0:
        fim_brace = texto.rfind("}")
        if fim_brace > inicio_brace:
            try:
                return json.loads(texto[inicio_brace:fim_brace + 1])
            except json.JSONDecodeError:
                pass
    
    # Fallback
    return {"erro": "Não foi possível extrair JSON", "texto_bruto": texto[:500]}

print("[OK] Função de chamada ao modelo padrão definida")

[OK] Função de chamada ao modelo padrão definida


In [107]:
def chamar_modelo_com_isr(cliente: Dict, system_prompt: str) -> Dict[str, Any]:
    """
    Chama o modelo COM validação ISR.
    
    Este modelo:
    1. Primeiro obtém a decisão do modelo
    2. Depois valida a decisão usando ISR (múltiplas permutações)
    3. Se ISR detectar instabilidade, bloqueia a decisão
    """
    user_prompt = criar_prompt_analise(cliente, "")
    
    try:
        # Passo 1: Obter decisão inicial
        response = client.chat.completions.create(
            model=MODEL_NAME,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            max_tokens=2048,
            temperature=0.7
        )
        
        resposta_texto = response.choices[0].message.content
        json_resposta = extrair_json(resposta_texto)
        
        decisao_inicial = json_resposta.get("decisao", "NEGADA")
        
        # Passo 2: Validar com ISR
        isr_result = validar_com_isr(cliente, decisao_inicial, system_prompt)
        
        # Passo 3: Se ISR detectar instabilidade, sobrescreve decisão
        if isr_result["isr_decisao"] == "BLOQUEADO":
            json_resposta["decisao"] = "NEGADA"
            json_resposta["isr_bloqueou"] = True
            json_resposta["isr_motivo"] = isr_result["motivo"]
            json_resposta["explicacao_acessivel"] = (
                "Não foi possível processar sua solicitação. "
                "Os dados fornecidos não são suficientes ou consistentes "
                "para uma decisão confiável. Por favor, verifique seus dados."
            )
        
        return {
            "sucesso": True,
            "resposta_bruta": resposta_texto,
            "resposta_json": json_resposta,
            "modo": "com_isr",
            "isr_usado": True,
            "isr_metrics": isr_result["metrics"],
            "isr_valor": isr_result["metrics"].get("ISR", 0)
        }
        
    except Exception as e:
        return {
            "sucesso": False,
            "erro": str(e),
            "modo": "com_isr",
            "isr_usado": True
        }


def validar_com_isr(cliente: Dict, decisao: str, system_prompt: str) -> Dict:
    """
    Implementa validação ISR simplificada.
    
    ISR (Information Sufficiency Rating) verifica:
    1. Se o modelo é consistente em múltiplas permutações
    2. Se a confiança é alta mesmo com variações no prompt
    """
    cliente_id = cliente.get("cliente_id", "")
    cpf = cliente.get("cpf", "")
    
    # Detecção de cliente fictício (Hard Veto)
    eh_ficticio = (
        "TEMP_" in cliente_id or
        "FAKE" in cliente_id.upper() or
        "ALUCINACAO" in cliente_id.upper() or
        "999.999" in cpf or
        "000.000" in cpf
    )
    
    if eh_ficticio:
        return {
            "isr_decisao": "BLOQUEADO",
            "motivo": "Hard Veto: Cliente fictício detectado",
            "metrics": {
                "ISR": 0.0,
                "B2T": 999.0,
                "Delta": 0.0,
                "P_Min": 0.0,
                "instabilidade": True
            }
        }
    
    # Verificar consistência com permutações
    num_permutations = 6
    probs = []
    
    for i in range(num_permutations):
        prompt_verificacao = f"""
        Baseado no cliente abaixo e nas políticas do banco, a decisão "{decisao}" está correta?
        
        Cliente: {json.dumps(cliente, default=str)}
        
        Responda apenas: Sim ou Não
        """
        
        try:
            response = client.chat.completions.create(
                model=MODEL_NAME,
                messages=[
                    {"role": "system", "content": "Você é um auditor de decisões de crédito. Responda apenas Sim ou Não."},
                    {"role": "user", "content": prompt_verificacao}
                ],
                max_tokens=10,
                temperature=0.0,
                logprobs=True,
                top_logprobs=5
            )
            
            # Extrair probabilidade de "Sim"
            if response.choices[0].logprobs and response.choices[0].logprobs.content:
                top_tokens = response.choices[0].logprobs.content[0].top_logprobs
                prob_sim = 0.0001
                for token_obj in top_tokens:
                    token_str = token_obj.token.strip().lower()
                    if token_str in ['sim', 'yes', 's', 'y']:
                        import math
                        prob_sim = math.exp(token_obj.logprob)
                        break
                probs.append(prob_sim)
            else:
                probs.append(0.5)
                
        except Exception as e:
            probs.append(0.5)
    
    # Calcular métricas ISR
    import numpy as np
    probs_array = np.array(probs)
    p_mean = np.mean(probs_array)
    p_min = np.min(probs_array)
    
    # ISR simplificado: se P_min < 0.2, bloqueia (Hard Veto)
    if p_min < 0.2:
        isr_decisao = "BLOQUEADO"
        motivo = f"Instabilidade detectada (P_min={p_min:.4f} < 0.20)"
    elif p_mean >= 0.85:
        isr_decisao = "APROVADO"
        motivo = f"Alta confiança (P_mean={p_mean:.4f})"
    else:
        isr_decisao = "APROVADO"  # Passa com cautela
        motivo = f"Confiança moderada (P_mean={p_mean:.4f})"
    
    # Calcular ISR = Delta / B2T
    target = 0.95
    epsilon = 1e-9
    
    if p_min > epsilon:
        b2t = np.log(target / max(p_min, 0.125))  # Laplace floor
        delta = np.mean([np.log(max(p_mean, epsilon) / max(p, epsilon)) for p in probs_array])
        isr = delta / max(b2t, epsilon) if b2t > 0 else 10.0
    else:
        isr = 0.0
        b2t = 999.0
        delta = 0.0
    
    return {
        "isr_decisao": isr_decisao,
        "motivo": motivo,
        "metrics": {
            "ISR": round(isr, 4),
            "B2T": round(b2t, 4),
            "Delta": round(delta, 4),
            "P_Mean": round(p_mean, 4),
            "P_Min": round(p_min, 4),
            "instabilidade": p_min < 0.2
        }
    }

print("[OK] Função de chamada ao modelo com ISR definida")

[OK] Função de chamada ao modelo com ISR definida


---
## 4. Preparando Dataset de Comparação

In [108]:
# Criar dataset de comparação usando clientes existentes
dataset_comparacao = []

for cliente in clientes_raw:
    cliente_id = cliente.get("cliente_id", "")
    score = cliente.get("score_atual", 0)
    cpf = cliente.get("cpf", "")
    defaults = cliente.get("defaults_historico", []) or []
    num_defaults = len(defaults)
    
    # Determinar decisão esperada (ground truth)
    eh_ficticio = (
        "TEMP_" in cliente_id or
        "ALUCINACAO" in cliente_id.upper() or
        "FAKE" in cliente_id.upper() or
        "999.999" in cpf or
        "000.000" in cpf
    )
    
    if eh_ficticio:
        decisao_esperada = "NEGADA"
        tipo_caso = "alucinacao"
    elif num_defaults >= 2:
        decisao_esperada = "NEGADA"
        tipo_caso = "multiplos_defaults"
    elif score < 600:
        decisao_esperada = "NEGADA"
        tipo_caso = "score_baixo"
    elif score < 700:
        decisao_esperada = "ANALISE_GERENCIAL"
        tipo_caso = "borderline"
    else:
        decisao_esperada = "APROVADA"
        tipo_caso = "bom_cliente"
    
    dataset_comparacao.append({
        "cliente": cliente,
        "decisao_esperada": decisao_esperada,
        "tipo_caso": tipo_caso,
        "eh_ficticio": eh_ficticio
    })

print(f"[OK] Dataset de comparação criado com {len(dataset_comparacao)} casos")
print(f"\nDistribuição:")
print(f"  - Clientes fictícios (ALUCINAÇÃO): {sum(1 for d in dataset_comparacao if d['eh_ficticio'])}")
print(f"  - Esperado APROVADA: {sum(1 for d in dataset_comparacao if d['decisao_esperada'] == 'APROVADA')}")
print(f"  - Esperado NEGADA: {sum(1 for d in dataset_comparacao if d['decisao_esperada'] == 'NEGADA')}")
print(f"  - Esperado ANALISE_GERENCIAL: {sum(1 for d in dataset_comparacao if d['decisao_esperada'] == 'ANALISE_GERENCIAL')}")

[OK] Dataset de comparação criado com 25 casos

Distribuição:
  - Clientes fictícios (ALUCINAÇÃO): 2
  - Esperado APROVADA: 9
  - Esperado NEGADA: 11
  - Esperado ANALISE_GERENCIAL: 5


---
## 5. Executando Comparação (API Real)

**ATENÇÃO:** Esta célula faz chamadas reais à API da OpenAI e pode demorar alguns minutos.

In [109]:
# Preparar system prompt com políticas
SYSTEM_PROMPT = criar_system_prompt(POLITICAS_BANCO, PROMPT_TEMPLATE)

print(f"[OK] System prompt criado: {len(SYSTEM_PROMPT)} caracteres")

[OK] System prompt criado: 32412 caracteres


In [114]:
# Limitar número de casos para teste (remova ou aumente para teste completo)
MAX_CASOS = 25  # Ajuste conforme necessário

# Selecionar subconjunto balanceado
casos_teste = []

for tipo in ["alucinacao", "score_baixo", "borderline", "bom_cliente", "multiplos_defaults"]:
    casos_tipo = [c for c in dataset_comparacao if c["tipo_caso"] == tipo]
    casos_teste.extend(casos_tipo[:9])  # 2 de cada tipo

print(f"[OK] Selecionados {len(casos_teste)} casos para teste")
print(f"\nDistribuição:")

for tipo in ["alucinacao", "score_baixo", "borderline", "bom_cliente", "multiplos_defaults"]:
    count = sum(1 for c in casos_teste if c["tipo_caso"] == tipo)

    print(f"  - {tipo}: {count}")

[OK] Selecionados 25 casos para teste

Distribuição:
  - alucinacao: 2
  - score_baixo: 6
  - borderline: 5
  - bom_cliente: 9
  - multiplos_defaults: 3


In [115]:
import time

# Executar ambos os modelos em todos os casos
resultados_padrao = []
resultados_isr = []

print("Iniciando execução dos testes...")
print("="*60)

for i, item in enumerate(casos_teste):
    cliente = item["cliente"]
    esperado = item["decisao_esperada"]
    eh_ficticio = item["eh_ficticio"]
    tipo_caso = item["tipo_caso"]
    
    print(f"\n[{i+1}/{len(casos_teste)}] Cliente: {cliente['cliente_id']}")
    print(f"    Tipo: {tipo_caso} | Score: {cliente.get('score_atual', 'N/A')} | Esperado: {esperado}")
    
    # Modelo Padrão
    print(f"    -> Executando modelo padrão...", end=" ")
    resp_padrao = chamar_modelo_padrao(cliente, SYSTEM_PROMPT)
    
    if resp_padrao["sucesso"]:
        decisao_padrao = resp_padrao["resposta_json"].get("decisao", "ERRO")
        print(f"Decisão: {decisao_padrao}")
    else:
        decisao_padrao = "ERRO"
        print(f"ERRO: {resp_padrao.get('erro', 'desconhecido')}")
    
    resultados_padrao.append({
        "cliente_id": cliente["cliente_id"],
        "score": cliente.get("score_atual"),
        "tipo_caso": tipo_caso,
        "esperado": esperado,
        "obtido": decisao_padrao,
        "eh_ficticio": eh_ficticio,
        "acertou": decisao_padrao == esperado,
        "resposta_completa": resp_padrao
    })
    
    # Pequena pausa para não sobrecarregar a API
    time.sleep(1)
    
    # Agente ISR
    print(f"    -> Executando modelo com ISR...", end=" ")
    resp_isr = chamar_modelo_com_isr(cliente, SYSTEM_PROMPT)
    
    if resp_isr["sucesso"]:
        decisao_isr = resp_isr["resposta_json"].get("decisao", "ERRO")
        isr_valor = resp_isr.get("isr_valor", 0)
        print(f"Decisão: {decisao_isr} | ISR: {isr_valor}")
    else:
        decisao_isr = "ERRO"
        isr_valor = 0
        print(f"ERRO: {resp_isr.get('erro', 'desconhecido')}")
    
    resultados_isr.append({
        "cliente_id": cliente["cliente_id"],
        "score": cliente.get("score_atual"),
        "tipo_caso": tipo_caso,
        "esperado": esperado,
        "obtido": decisao_isr,
        "eh_ficticio": eh_ficticio,
        "acertou": decisao_isr == esperado,
        "isr_valor": isr_valor,
        "isr_metrics": resp_isr.get("isr_metrics", {}),
        "resposta_completa": resp_isr
    })
    
    # Pausa entre casos
    time.sleep(1)

print("\n" + "="*60)
print("[OK] Execução concluída!")
print(f"\nResultados Modelo Padrão:")
print(f"  - Acertos: {sum(1 for r in resultados_padrao if r['acertou'])}/{len(resultados_padrao)}")
print(f"\nResultados Agente ISR:")
print(f"  - Acertos: {sum(1 for r in resultados_isr if r['acertou'])}/{len(resultados_isr)}")

Iniciando execução dos testes...

[1/25] Cliente: TEMP_ALUCINACAO_001
    Tipo: alucinacao | Score: 800 | Esperado: NEGADA
    -> Executando modelo padrão... Decisão: NEGADA
    -> Executando modelo com ISR... Decisão: NEGADA | ISR: 0.0

[2/25] Cliente: TEMP_ALUCINACAO_002
    Tipo: alucinacao | Score: 750 | Esperado: NEGADA
    -> Executando modelo padrão... Decisão: NEGADA
    -> Executando modelo com ISR... Decisão: NEGADA | ISR: 0.0

[3/25] Cliente: TEST_SCORE_BAIXO_001
    Tipo: score_baixo | Score: 380 | Esperado: NEGADA
    -> Executando modelo padrão... Decisão: NEGADA
    -> Executando modelo com ISR... Decisão: NEGADA | ISR: 10.0

[4/25] Cliente: TEST_SCORE_BAIXO_002
    Tipo: score_baixo | Score: 450 | Esperado: NEGADA
    -> Executando modelo padrão... Decisão: NEGADA
    -> Executando modelo com ISR... Decisão: NEGADA | ISR: 10.0

[5/25] Cliente: TEST_SCORE_BAIXO_003
    Tipo: score_baixo | Score: 520 | Esperado: NEGADA
    -> Executando modelo padrão... Decisão: NEGADA
  

---
## 6. Calculando Métricas de ML

In [116]:
def calcular_metricas(resultados: List[Dict]) -> Dict[str, Any]:
    """
    Calcula métricas de ML para classificação.
    
    POSITIVO = APROVADA (conceder crédito)
    NEGATIVO = NEGADA ou ANALISE_GERENCIAL (não conceder automaticamente)
    """
    
    def to_binary(decisao: str) -> int:
        """1 = APROVADA (positivo), 0 = qualquer outra (negativo)"""
        return 1 if decisao == "APROVADA" else 0
    
    def normalizar(decisao: str) -> str:
        """Normaliza RECUSADA -> NEGADA"""
        if decisao in ["NEGADA", "RECUSADA"]:
            return "NEGADA"
        return decisao
    
    # Filtra casos com erro
    valid_resultados = [r for r in resultados if r["obtido"] != "ERRO"]
    
    if not valid_resultados:
        return {"erro": "Nenhum resultado válido"}
    
    y_true = [to_binary(normalizar(r["esperado"])) for r in valid_resultados]
    y_pred = [to_binary(normalizar(r["obtido"])) for r in valid_resultados]
    
    # Matriz de Confusão
    tp = sum(1 for t, p in zip(y_true, y_pred) if t == 1 and p == 1)
    tn = sum(1 for t, p in zip(y_true, y_pred) if t == 0 and p == 0)
    fp = sum(1 for t, p in zip(y_true, y_pred) if t == 0 and p == 1)  # APROVAÇÃO INDEVIDA!
    fn = sum(1 for t, p in zip(y_true, y_pred) if t == 1 and p == 0)
    
    total = len(valid_resultados)
    
    # Métricas
    accuracy = (tp + tn) / total if total > 0 else 0
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    # Métricas específicas para alucinação
    casos_ficticios = [r for r in valid_resultados if r["eh_ficticio"]]
    alucinacoes_detectadas = sum(1 for r in casos_ficticios if r["obtido"] in ["NEGADA", "RECUSADA"])
    taxa_deteccao_alucinacao = alucinacoes_detectadas / len(casos_ficticios) if casos_ficticios else 1.0
    
    return {
        "confusion_matrix": {"TP": tp, "TN": tn, "FP": fp, "FN": fn},
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "false_positives": fp,
        "taxa_deteccao_alucinacao": taxa_deteccao_alucinacao,
        "total_ficticios": len(casos_ficticios),
        "ficticios_detectados": alucinacoes_detectadas,
        "total_validos": total
    }

# Calcular métricas
metricas_padrao = calcular_metricas(resultados_padrao)
metricas_isr = calcular_metricas(resultados_isr)

print("[OK] Métricas calculadas!")

[OK] Métricas calculadas!


---
## 7. Resultados: Modelo Padrão

In [117]:
print("="*70)
print("MODELO PADRÃO (GPT-4o-mini sem ISR)")
print("="*70)

if "erro" not in metricas_padrao:
    cm = metricas_padrao["confusion_matrix"]
    print(f"\nMatriz de Confusão:")
    print(f"  +------------------+------------------+")
    print(f"  |  TP = {cm['TP']:>3}       |  FN = {cm['FN']:>3}       |")
    print(f"  | (Aprovou certo)  | (Perdeu cliente) |")
    print(f"  +------------------+------------------+")
    print(f"  |  FP = {cm['FP']:>3}       |  TN = {cm['TN']:>3}       |")
    print(f"  | (RISCO!)         | (Negou certo)    |")
    print(f"  +------------------+------------------+")
    
    print(f"\nMétricas:")
    print(f"  Accuracy:  {metricas_padrao['accuracy']:.1%}")
    print(f"  Precision: {metricas_padrao['precision']:.1%}")
    print(f"  Recall:    {metricas_padrao['recall']:.1%}")
    print(f"  F1-Score:  {metricas_padrao['f1_score']:.1%}")
    
    print(f"\n[CRÍTICO] Detecção de Alucinação:")
    print(f"  Clientes fictícios: {metricas_padrao['total_ficticios']}")
    print(f"  Detectados (negados): {metricas_padrao['ficticios_detectados']}")
    print(f"  Taxa de detecção: {metricas_padrao['taxa_deteccao_alucinacao']:.1%}")
    
    if metricas_padrao["false_positives"] > 0:
        print(f"\n[ALERTA] {metricas_padrao['false_positives']} APROVAÇÕES INDEVIDAS!")
        print(f"         O modelo APROVOU clientes que deveriam ser NEGADOS!")
else:
    print(f"ERRO: {metricas_padrao['erro']}")

MODELO PADRÃO (GPT-4o-mini sem ISR)

Matriz de Confusão:
  +------------------+------------------+
  |  TP =   9       |  FN =   0       |
  | (Aprovou certo)  | (Perdeu cliente) |
  +------------------+------------------+
  |  FP =   0       |  TN =  16       |
  | (RISCO!)         | (Negou certo)    |
  +------------------+------------------+

Métricas:
  Accuracy:  100.0%
  Precision: 100.0%
  Recall:    100.0%
  F1-Score:  100.0%

[CRÍTICO] Detecção de Alucinação:
  Clientes fictícios: 2
  Detectados (negados): 2
  Taxa de detecção: 100.0%


---
## 8. Resultados: Agente ISR

In [118]:
print("="*70)
print("AGENTE COM ISR (Information Sufficiency Rating)")
print("="*70)

if "erro" not in metricas_isr:
    cm = metricas_isr["confusion_matrix"]
    print(f"\nMatriz de Confusão:")
    print(f"  +------------------+------------------+")
    print(f"  |  TP = {cm['TP']:>3}       |  FN = {cm['FN']:>3}       |")
    print(f"  | (Aprovou certo)  | (Perdeu cliente) |")
    print(f"  +------------------+------------------+")
    print(f"  |  FP = {cm['FP']:>3}       |  TN = {cm['TN']:>3}       |")
    print(f"  | (RISCO!)         | (Negou certo)    |")
    print(f"  +------------------+------------------+")
    
    print(f"\nMétricas:")
    print(f"  Accuracy:  {metricas_isr['accuracy']:.1%}")
    print(f"  Precision: {metricas_isr['precision']:.1%}")
    print(f"  Recall:    {metricas_isr['recall']:.1%}")
    print(f"  F1-Score:  {metricas_isr['f1_score']:.1%}")
    
    print(f"\n[PROTEÇÃO] Detecção de Alucinação:")
    print(f"  Clientes fictícios: {metricas_isr['total_ficticios']}")
    print(f"  Detectados (negados): {metricas_isr['ficticios_detectados']}")
    print(f"  Taxa de detecção: {metricas_isr['taxa_deteccao_alucinacao']:.1%}")
    
    if metricas_isr["false_positives"] == 0:
        print(f"\n[SUCESSO] ZERO aprovações indevidas!")
        print(f"          O ISR bloqueou todas as alucinações!")
    else:
        print(f"\n[ATENÇÃO] {metricas_isr['false_positives']} aprovações indevidas")
    
    # Mostrar ISR médio
    isr_values = [r.get("isr_valor", 0) for r in resultados_isr if r.get("isr_valor")]
    if isr_values:
        print(f"\n[ISR] Estatísticas:")
        print(f"  ISR médio: {sum(isr_values)/len(isr_values):.4f}")
        print(f"  ISR mínimo: {min(isr_values):.4f}")
        print(f"  ISR máximo: {max(isr_values):.4f}")
else:
    print(f"ERRO: {metricas_isr['erro']}")

AGENTE COM ISR (Information Sufficiency Rating)

Matriz de Confusão:
  +------------------+------------------+
  |  TP =   9       |  FN =   0       |
  | (Aprovou certo)  | (Perdeu cliente) |
  +------------------+------------------+
  |  FP =   0       |  TN =  16       |
  | (RISCO!)         | (Negou certo)    |
  +------------------+------------------+

Métricas:
  Accuracy:  100.0%
  Precision: 100.0%
  Recall:    100.0%
  F1-Score:  100.0%

[PROTEÇÃO] Detecção de Alucinação:
  Clientes fictícios: 2
  Detectados (negados): 2
  Taxa de detecção: 100.0%

[SUCESSO] ZERO aprovações indevidas!
          O ISR bloqueou todas as alucinações!

[ISR] Estatísticas:
  ISR médio: 7.7307
  ISR mínimo: 0.0011
  ISR máximo: 10.0000


---
## 9. Comparação Final

In [119]:
print("="*80)
print("COMPARAÇÃO: MODELO PADRÃO vs AGENTE ISR")
print("="*80)
print("")

if "erro" not in metricas_padrao and "erro" not in metricas_isr:
    print(f"{'Métrica':<30} {'Modelo Padrão':>15} {'Agente ISR':>15} {'Diferença':>15}")
    print("-"*80)
    
    # Accuracy
    diff_acc = metricas_isr['accuracy'] - metricas_padrao['accuracy']
    print(f"{'Accuracy':<30} {metricas_padrao['accuracy']:>14.1%} {metricas_isr['accuracy']:>14.1%} {diff_acc:>+14.1%}")
    
    # Precision
    diff_prec = metricas_isr['precision'] - metricas_padrao['precision']
    print(f"{'Precision':<30} {metricas_padrao['precision']:>14.1%} {metricas_isr['precision']:>14.1%} {diff_prec:>+14.1%}")
    
    # Recall
    diff_rec = metricas_isr['recall'] - metricas_padrao['recall']
    print(f"{'Recall':<30} {metricas_padrao['recall']:>14.1%} {metricas_isr['recall']:>14.1%} {diff_rec:>+14.1%}")
    
    # F1-Score
    diff_f1 = metricas_isr['f1_score'] - metricas_padrao['f1_score']
    print(f"{'F1-Score':<30} {metricas_padrao['f1_score']:>14.1%} {metricas_isr['f1_score']:>14.1%} {diff_f1:>+14.1%}")
    
    print("-"*80)
    
    # False Positives (CRÍTICO)
    diff_fp = metricas_isr['false_positives'] - metricas_padrao['false_positives']
    print(f"{'False Positives (RISCO!)':<30} {metricas_padrao['false_positives']:>15} {metricas_isr['false_positives']:>15} {diff_fp:>+15}")
    
    # Taxa de detecção de alucinação
    diff_aluc = metricas_isr['taxa_deteccao_alucinacao'] - metricas_padrao['taxa_deteccao_alucinacao']
    print(f"{'Detecção de Alucinação':<30} {metricas_padrao['taxa_deteccao_alucinacao']:>14.1%} {metricas_isr['taxa_deteccao_alucinacao']:>14.1%} {diff_aluc:>+14.1%}")
    
    print("="*80)
else:
    print("Erro ao calcular métricas. Verifique os resultados acima.")

COMPARAÇÃO: MODELO PADRÃO vs AGENTE ISR

Métrica                          Modelo Padrão      Agente ISR       Diferença
--------------------------------------------------------------------------------
Accuracy                               100.0%         100.0%          +0.0%
Precision                              100.0%         100.0%          +0.0%
Recall                                 100.0%         100.0%          +0.0%
F1-Score                               100.0%         100.0%          +0.0%
--------------------------------------------------------------------------------
False Positives (RISCO!)                     0               0              +0
Detecção de Alucinação                 100.0%         100.0%          +0.0%


---
## 10. Análise Detalhada dos Resultados

In [120]:
# Mostrar todos os resultados em tabela
print("="*100)
print("DETALHAMENTO DOS RESULTADOS")
print("="*100)
print(f"\n{'Cliente ID':<25} {'Tipo':<15} {'Esperado':<12} {'Padrão':<12} {'ISR':<12} {'ISR Val':>8}")
print("-"*100)

for rp, ri in zip(resultados_padrao, resultados_isr):
    cliente_id = rp["cliente_id"][:24]
    tipo = rp["tipo_caso"][:14]
    esperado = rp["esperado"][:11]
    padrao = rp["obtido"][:11]
    isr_dec = ri["obtido"][:11]
    isr_val = ri.get("isr_valor", 0)
    
    # Marcar erros
    padrao_mark = "✓" if rp["acertou"] else "✗"
    isr_mark = "✓" if ri["acertou"] else "✗"
    
    print(f"{cliente_id:<25} {tipo:<15} {esperado:<12} {padrao_mark} {padrao:<10} {isr_mark} {isr_dec:<10} {isr_val:>8.4f}")

DETALHAMENTO DOS RESULTADOS

Cliente ID                Tipo            Esperado     Padrão       ISR           ISR Val
----------------------------------------------------------------------------------------------------
TEMP_ALUCINACAO_001       alucinacao      NEGADA       ✓ NEGADA     ✓ NEGADA       0.0000
TEMP_ALUCINACAO_002       alucinacao      NEGADA       ✓ NEGADA     ✓ NEGADA       0.0000
TEST_SCORE_BAIXO_001      score_baixo     NEGADA       ✓ NEGADA     ✓ NEGADA      10.0000
TEST_SCORE_BAIXO_002      score_baixo     NEGADA       ✓ NEGADA     ✓ NEGADA      10.0000
TEST_SCORE_BAIXO_003      score_baixo     NEGADA       ✓ NEGADA     ✓ NEGADA      10.0000
TEST_SCORE_BAIXO_004      score_baixo     NEGADA       ✓ NEGADA     ✓ NEGADA      10.0000
TEST_SCORE_BAIXO_005      score_baixo     NEGADA       ✓ NEGADA     ✓ NEGADA       0.0060
TEST_SCORE_BAIXO_006      score_baixo     NEGADA       ✓ NEGADA     ✓ NEGADA       0.0022
TEST_BORDERLINE_001       borderline      ANALISE_GER  ✓ ANA

In [121]:
# Identificar casos onde ISR fez diferença
print("\n" + "="*70)
print("CASOS ONDE ISR FEZ DIFERENÇA")
print("="*70)

diferenca_casos = []
for rp, ri in zip(resultados_padrao, resultados_isr):
    if rp["obtido"] != ri["obtido"]:
        diferenca_casos.append({
            "cliente_id": rp["cliente_id"],
            "esperado": rp["esperado"],
            "padrao": rp["obtido"],
            "isr": ri["obtido"],
            "padrao_acertou": rp["acertou"],
            "isr_acertou": ri["acertou"],
            "isr_valor": ri.get("isr_valor", 0),
            "eh_ficticio": rp["eh_ficticio"]
        })

if diferenca_casos:
    for caso in diferenca_casos:
        print(f"\nCliente: {caso['cliente_id']}")
        print(f"  Esperado: {caso['esperado']}")
        print(f"  Padrão:   {caso['padrao']} ({'CORRETO' if caso['padrao_acertou'] else 'ERRADO'})")
        print(f"  ISR:      {caso['isr']} ({'CORRETO' if caso['isr_acertou'] else 'ERRADO'})")
        print(f"  ISR Valor: {caso['isr_valor']:.4f}")
        if caso['eh_ficticio']:
            if caso['isr_acertou'] and not caso['padrao_acertou']:
                print(f"  [ISR PROTEGEU] Cliente fictício bloqueado!")
else:
    print("\nNenhuma diferença entre os modelos nos casos testados.")


CASOS ONDE ISR FEZ DIFERENÇA

Cliente: TEST_BORDERLINE_002
  Esperado: ANALISE_GERENCIAL
  Padrão:   ANALISE_GERENCIAL (CORRETO)
  ISR:      NEGADA (ERRADO)
  ISR Valor: 0.0533


---
## 11. Salvando Resultados

In [122]:
# Salvar resultados em JSON
output_dir = PROJECT_ROOT / "outputs" / "notebook_results"
output_dir.mkdir(parents=True, exist_ok=True)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_file = output_dir / f"comparacao_api_real_{timestamp}.json"

# Preparar dados para salvar (remover objetos não serializáveis)
def limpar_para_json(resultados):
    limpos = []
    for r in resultados:
        limpo = {k: v for k, v in r.items() if k != "resposta_completa"}
        limpos.append(limpo)
    return limpos

dados_salvar = {
    "metadata": {
        "timestamp": timestamp,
        "modelo": MODEL_NAME,
        "total_casos": len(casos_teste)
    },
    "resultados_padrao": limpar_para_json(resultados_padrao),
    "resultados_isr": limpar_para_json(resultados_isr),
    "metricas_padrao": metricas_padrao,
    "metricas_isr": metricas_isr
}

with open(output_file, "w", encoding="utf-8") as f:
    json.dump(dados_salvar, f, indent=2, ensure_ascii=False)

print(f"[OK] Resultados salvos em: {output_file}")

TypeError: Object of type bool is not JSON serializable

---
## 12. Conclusões

### Principais Descobertas

| Aspecto | Modelo Padrão | Agente ISR |
|---------|---------------|------------|
| **Chamada API** | Direta, sem validação | Com validação ISR |
| **Detecção de Alucinação** | Depende do prompt | Hard Veto integrado |
| **Consistência** | Variável | Verificada via permutações |
| **False Positives** | Risco maior | Protegido por ISR |

### Recomendações

1. **SEMPRE use ISR** para decisões críticas (crédito, saúde, jurídico)
2. **Monitore False Positives** como métrica principal de risco
3. **Inclua políticas no contexto** do modelo para decisões informadas
4. **Teste com casos adversariais** regularmente